## Ensemble Learning: Get a prediction by using multiple models and taking a majority vote. Most common ensemble learning methods: Bagging and Boosting.

**Bagging (Bootstrap Aggregation):**<br>
- Create 'n' subsets using sampling with replacement (can get same data point multiple times).
- Create model for each sample subset.
- Make a prediction using each model.
- Take a majority vote and use the result as your final prediction.
- This works for classification problems, but also works for regression (continuous variable output) by taking an average of the predictions by each model.
- Process of creating subsets is called bootstrapping, process of averaging each model's prediction is called aggregation.

**Random Forest Algorithm:**<br>
- Bagged Tree technique.
- Features are sampled in addition to dataset.
- Train a decision tree on each each sample subset, and aggregate the predictions to come up with a final prediction.

**Bagging vs Bagged Trees:**<br>
- Bagging: Underlying model can be anything (SVM, KNN, Logistic Regression, etc.)
- Bagged Trees: Each model is a tree.

## Import dataset.

In [2]:
import pandas as pd

df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [4]:
df.Outcome.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

## Train/Test Splits. Here we use 'stratify=y' to be safe because there is slight (2:1) imbalance in our target column. Stratify will maintain the 2:1 ratio in it's random samples.

In [5]:
x = df.drop("Outcome",axis="columns")
y = df.Outcome

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [13]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, stratify=y, random_state=10)

## Train using standalone model (5 fold). We get accuracy score of 71%.

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

scores = cross_val_score(DecisionTreeClassifier(), x, y, cv=5)
scores.mean()

0.712375859434683

## Train using Bagging. We get accuracy score of 77% and cross-validated score of 75% (5 fold).
## Out of Bag (OOB): Treat data points that are not randomly selected as your testing sample.

In [15]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
bag_model.fit(x_train, y_train)
bag_model.oob_score_

0.7534722222222222

In [16]:
bag_model.score(x_test, y_test)

0.7760416666666666

In [18]:
bag_model = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=100, 
    max_samples=0.8, 
    oob_score=True,
    random_state=0
)
scores = cross_val_score(bag_model, x, y, cv=5)
scores.mean()

0.7578728461081402

## Train using Random Forest. We get cross-validated score of 76% (5 fold).

In [19]:
from sklearn.ensemble import RandomForestClassifier

scores = cross_val_score(RandomForestClassifier(n_estimators=50), x, y, cv=5)
scores.mean()

0.7617604617604619